# Model training #1

For testing purposes only, on the simulated dataset, file savings are commented out.

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import xgboost
from sklearn.ensemble import RandomForestClassifier

# tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import AlphaDropout, Conv1D, Flatten
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
tf.test.is_gpu_available(cuda_only=True)

### Data transformation/split:

In [ ]:
featuretype = pd.read_csv('feature_types_final_simul.csv', sep=';')
df = pd.read_csv('data_clean_onehot_final_simulated.csv', sep=';')

# preprocess
df = df.sample(frac=1) # shuffle the data beforehand
dataset = df.dropna().values

# test and validation ratio
test_step = .1

# features and output
X = dataset[:,0:-1]
Y = dataset[:,-1]

# indexing
test_index = int(X.shape[0]*(1-test_step))

# test split
X_test = X[test_index:]
Y_test = Y[test_index:]
# Y_test = to_categorical(Y[test_index:],2)

# training split
X_train = X[:test_index]
Y_train = Y[:test_index]
# Y_train = to_categorical(Y[:test_index],2)


# MinMax scaling

scaler = MinMaxScaler(feature_range=(0, 1)).fit(X_train)
x_train = scaler.transform(X_train)
x_test = scaler.transform(X_test)
y_train = Y_train.astype(int)
y_test = Y_test.astype(int)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

### Build the tf.keras.Sequential model by stacking layers.

Defaults are based on the hyper-parameter optimization.


In [ ]:
def create_model(in_dim, dense_layer_sizes = 24, dropout_ratios = .4, depth = 7,
                 kernin = 'glorot_uniform', opti = 'adam', activ= 'selu'):
    
    model = Sequential()
    model.add(Dense(dense_layer_sizes,
                    activation=activ,
                    kernel_initializer=kernin,
                    input_shape=(in_dim,)))
    
#     note: for testing Conv1D as first layer (ditched since no visible improvement)
#     model.add(Conv1D(6, (6), activation=activ, input_shape=(32,1)))
#     model.add(Flatten())

    for i in range(depth-1): 
        model.add(Dropout(dropout_ratios))
        # model.add(BatchNormalization())
        # model.add(AlphaDropout(dropout_ratios))
        model.add(Dense(dense_layer_sizes,
                        activation=activ,
                        kernel_initializer=kernin))
        
    model.add(Dense(1, activation='sigmoid',
                    kernel_initializer=kernin)) # softmax if the output is shaped as 2-dim
    
    model.compile(optimizer=opti,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


Cross-validation training with callbacks:

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=40, verbose=1)
redlrplat = tf.keras.callbacks.ReduceLROnPlateau(patience=20, verbose=1)
#checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='model1.hdf5', save_best_only=True, verbose=1)


# note: for testing Conv1D as first layer (ditched since no visible improvement)
# x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
# x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1],1)

skf = StratifiedKFold(n_splits=3, shuffle=True)

cross_histories = []
for train_index, valid_index in skf.split(x_train, y_train):
    model = create_model(in_dim = x_train.shape[1])
    
    # Train and evaluate model:
    network_history=model.fit(x_train[train_index], y_train[train_index], 
                batch_size=16, 
                epochs=10, 
                verbose=1,
                shuffle=True,
                validation_data=(x_train[valid_index], y_train[valid_index]), 
                callbacks=[early_stopping, redlrplat])
    cross_histories.append(network_history)

In [ ]:
# print("Evaluation")
# model.evaluate(x_test, Y_test)

def plot_history(network_histories, fp=None):
    plt.figure(figsize=(20,4))
    plt.subplot(2,2,1)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    for network_history in network_histories:
        plt.plot(network_history.history['loss'])
    plt.legend(['Training'])

    plt.subplot(2,2,2)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    for network_history in network_histories:
        plt.plot(network_history.history['acc'])
    plt.legend(['Training'], loc='lower right')

    plt.subplot(2,2,3)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    for network_history in network_histories:
        plt.plot(network_history.history['val_loss'])
    plt.legend(['Validation'])

    plt.subplot(2,2,4)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    for network_history in network_histories:
        plt.plot(network_history.history['val_acc'])
    plt.legend(['Validation'], loc='lower right')
    
#    plt.savefig(fp, bbox_inches='tight')
    plt.show()

In [ ]:
plot_history(cross_histories)#, 'model1_cross.pdf')

In [ ]:
# hhh= [ h.history for h in cross_histories ]
# pickle.dump( hhh, open( "cv_histories_1.p", "wb" ) )

## Alternative network structure

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=40, verbose=1)
redlrplat = tf.keras.callbacks.ReduceLROnPlateau(patience=20, verbose=1)
#checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='model1.hdf5', save_best_only=True, verbose=1)


# note: for testing Conv1D as first layer (ditched since no visible improvement)
# x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
# x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1],1)

skf = StratifiedKFold(n_splits=5, shuffle=True)

cross_histories2 = []
for train_index, valid_index in skf.split(x_train, y_train):
    model2 = create_model(in_dim = x_train.shape[1],
                         dense_layer_sizes = 40, dropout_ratios = .4, depth = 6,
                      kernin = 'glorot_normal', opti = 'rmsprop', activ= 'relu')
    
    # Train and evaluate model:
    network_history2=model2.fit(x_train[train_index], y_train[train_index], 
                batch_size=16, 
                epochs=10, 
                verbose=1,
                shuffle=True,
                validation_data=(x_train[valid_index], y_train[valid_index]), 
                callbacks=[early_stopping, redlrplat])
    cross_histories2.append(network_history2)

In [ ]:
plot_history(cross_histories2)#, 'model2_cross.pdf')

In [ ]:
# hhh2= [ h.history for h in cross_histories2 ]
# pickle.dump( hhh2, open( "cv_histories_2.p", "wb" ) )

# Final model

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=60, verbose=1)
redlrplat = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=30, verbose=1)
# checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='model_final.hdf5', save_best_only=True, verbose=1)


my_classifier = KerasClassifier(create_model,
                                in_dim = x_train.shape[1],
                                batch_size=16, 
                                epochs=50, 
                                verbose=1,
                                shuffle=True,
                                validation_data=(x_test, y_test), 
                                callbacks=[early_stopping, redlrplat, checkpointer])

fitted_history = my_classifier.fit(x_train, y_train)

# model = create_model(in_dim = x_train.shape[1])
# # Train and evaluate model:
# network_history=model.fit(x_train, y_train, 
#             batch_size=16, 
#             epochs=300, 
#             verbose=1,
#             shuffle=True,
#             validation_data=(x_test, y_test), 
#             callbacks=[early_stopping, redlrplat, checkpointer])

In [ ]:
# my_classifier.model.save("model_final2.hdf5")

In [ ]:
def plot_history_onemodel(network_history, fp=None):
    plt.figure(figsize=(20,4))
    plt.subplot(1,2,1)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(network_history.history['loss'])
    plt.plot(network_history.history['val_loss'])
    plt.legend(['Training', 'Test'])

    plt.subplot(1,2,2)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(network_history.history['acc'])
    plt.plot(network_history.history['val_acc'])
    plt.legend(['Training', 'Test'], loc='lower right')
    
#     plt.savefig(fp, bbox_inches='tight')
    plt.show()

In [ ]:
plot_history_onemodel(fitted_history)#, "final_model.pdf")

## XGBoost
for comparison with the NN model

In [ ]:
gbtree = xgboost.XGBClassifier(n_estimators=100, max_depth=5)
gbtree.fit(x_train, Y_train)

print(accuracy_score(Y_train, gbtree.predict(x_train)))
print(accuracy_score(Y_test, gbtree.predict(x_test)))

## RandomForest
for comparison with the NN model

In [ ]:
rfmod = RandomForestClassifier(n_estimators=100, max_depth= 5)
rfmod.fit(x_train, y_train)

print(rfmod.score(x_train,y_train))
print(rfmod.score(x_test, y_test))

reload the NN model:

In [ ]:
# finmod = tf.keras.models.load_model("model_final2.hdf5")
finmod = my_classifier.model

In [ ]:
finmod.summary()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

def roccc(model, modelname, tfkeras = False):
    if tfkeras:
        y_hat = model.predict(x_test)
    else:
        y_hat = model.predict_proba(x_test)[:,1]
        

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    fpr[0], tpr[0], _ = roc_curve(y_test, y_hat)
    roc_auc[0] = auc(fpr[0], tpr[0])

#     # Compute micro-average ROC curve and ROC area
#     fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_hat.ravel())
#     roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    lw = 2
    ROC = plt.plot(fpr[0], tpr[0],
         lw=lw, label = modelname + ' ROC (area = %0.2f)' % roc_auc[0])
    
    return fpr, tpr, roc_auc

In [ ]:
plt.figure(figsize=(5,5))

roccc(finmod, tfkeras=True, modelname = 'DNN')
roccc(gbtree, modelname = 'XGBoost')
roccc(rfmod, modelname = 'RandFor')

plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")

# plt.savefig('roc_compa.pdf', bbox_inches='tight')
plt.show()

# Bagging estimator

In [ ]:
from sklearn.ensemble import BaggingClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def create_sklearn_model():
#     dens = np.random.choice([16,24,32,40])
#     dep = np.random.choice([3,4,5,6,7])
#     act = np.random.choice(['selu', 'relu', 'sigmoid'])
#     drop =np.random.choice([.2,.3,.4,.5])
    
    my_classifier = KerasClassifier(create_model,
                                    in_dim=x_train.shape[1],
                                    dense_layer_sizes = 24,
                                    dropout_ratios = .4,
                                    depth = 7,
                                    activ= 'selu',
                                batch_size=16, 
                                epochs=50, 
                                verbose=1,
                                shuffle=True)
    return my_classifier


bagclf = BaggingClassifier(base_estimator=create_sklearn_model(),
                           n_estimators=10,
                           random_state=0,
                          bootstrap=True)

bagclf.fit(x_train, y_train)

In [ ]:
bagclf.score(x_test, y_test)

In [ ]:
from sklearn.metrics import plot_roc_curve

bagclf_disp = plot_roc_curve(bagclf, x_train, y_train)
bagclf_disp.figure_.suptitle("ROC curve comparison")

plt.show()